In [1]:
import pickle
import numpy as np

In [2]:
with open('fulldata', 'rb') as f: 
    data= pickle.load(f)

In [3]:
print(data.keys())

dict_keys(['disgust', 'happy', 'neutral', 'surprise', 'sad', 'angry', 'fear'])


In [4]:
print(type(data['sad']))
print(np.min(data['sad']))

<class 'numpy.ndarray'>
-771.9886578677832


In [5]:
x=data['disgust']
for i in data.keys():
    if i!='disgust':
        x=np.append(x,data[i],axis=0)


In [6]:
x.shape

(2800, 2580)

In [8]:
y=np.zeros(2800)
y[:400]=0
y[400:800]=1
y[800:1200]=2
y[1200:1600]=3
y[1600:2000]=4
y[2000:2400]=5
y[2400:2800]=6
from keras.utils import np_utils
y = np_utils.to_categorical(y)

c:\users\swaraj\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
print(type(y))
print(y.shape)

<class 'numpy.ndarray'>
(2800, 7)


In [10]:
from sklearn.utils import shuffle
x,y=shuffle(x,y,random_state=2)

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=25)

In [12]:
from keras.models import Model
from keras.layers import Input, Conv1D, Dropout, MaxPooling1D, Flatten, Dense
from keras import regularizers

a = Input(shape=(2580,1,))
b = Conv1D(16, 15, activation='relu')(a)
b= MaxPooling1D(6)(b)
# b= Dropout(0.3)(b)

b =Conv1D(32, 5, activation='relu')(b)
b= MaxPooling1D(5)(b)
# b= Dropout(0.3)(b)

b = Conv1D(64, 3, activation='relu')(b)
b= MaxPooling1D(4)(b)
# b= Dropout(0.3)(b)

b = Conv1D(128, 3, activation='relu')(b)
b= MaxPooling1D(3)(b)
# b= Dropout(0.3)(b)

b= Flatten()(b)
# b = Dense(1024, activation='relu')(b)
# b= Dropout(0.2)(b)
b = Dense(128, activation='relu')(b)
# b= Dropout(0.2)(b)
b = Dense(7,activation='softmax')(b)
emotion= Model(inputs=a, outputs=b)

In [13]:
emotion.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2580, 1)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2566, 16)          256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 427, 16)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 423, 32)           2592      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 84, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 82, 64)            6208      
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 20, 64)            0         
__________

In [15]:
from keras.optimizers import Adam
optimizer =Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

In [17]:
emotion.compile(optimizer=optimizer, loss='categorical_crossentropy')

In [18]:
x= x.reshape((2800,2580,1))

In [19]:
emotion.fit(x, y,
                epochs=10,
                batch_size=64,
                shuffle=True,
                validation_data=(x[2000:], y[2000:])
               )

Train on 2800 samples, validate on 800 samples
Epoch 1/10
2800/2800 [==============================] - 25s - loss: 8.5131 - val_loss: 1.6445
Epoch 2/10
2800/2800 [==============================] - 2s - loss: 0.8267 - val_loss: 0.2838
Epoch 3/10
2800/2800 [==============================] - 3s - loss: 0.2242 - val_loss: 0.0741
Epoch 4/10
2800/2800 [==============================] - 2s - loss: 0.0925 - val_loss: 0.0662
Epoch 5/10
2800/2800 [==============================] - 2s - loss: 0.0546 - val_loss: 0.0171
Epoch 6/10
2800/2800 [==============================] - 2s - loss: 0.0200 - val_loss: 0.0120
Epoch 7/10
2800/2800 [==============================] - 2s - loss: 0.0140 - val_loss: 0.0095
Epoch 8/10
2800/2800 [==============================] - 2s - loss: 0.0076 - val_loss: 0.0048
Epoch 9/10
2800/2800 [==============================] - 2s - loss: 0.0036 - val_loss: 0.0039
Epoch 10/10
2800/2800 [==============================] - 3s - loss: 0.0044 - val_loss: 0.0044


In [20]:
# serialize model to JSON
model_json = emotion.to_json()
with open("model_conv.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
emotion.save_weights("model_conv.h5")
print("Saved model to disk")

Saved model to disk


In [21]:
y_pred1=emotion.predict(data['surprise'].reshape(400,2580,1))
np.argmax(y_pred1,axis=1)

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,